In [1]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import pickle
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

2025-05-16 19:43:50.449168: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 19:43:50.451183: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-16 19:43:50.482145: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-16 19:43:50.482184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-16 19:43:50.483088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
model = load_model('model.h5')

2025-05-16 19:43:52.864697: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
with open('label_encoding_gender.pkl', 'rb') as file:
    label_encoding_gender = pickle.load(file)

with open('onehot_encoding_geo.pkl', 'rb') as file:
    onehot_encoding_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [4]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
main_df = pd.DataFrame([input_data])

In [6]:
main_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
main_df['Gender'] = label_encoding_gender.transform(main_df['Gender'])

In [8]:
values = onehot_encoding_geo.transform(main_df[['Geography']])

onehot_encoding_df = pd.DataFrame(values.toarray(), columns=onehot_encoding_geo.get_feature_names_out())

main_df = pd.concat([main_df, onehot_encoding_df], axis = 1).drop(columns = ['Geography'])

In [9]:
main_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
scaler_values = scaler.transform(main_df)

In [14]:
prediction=model.predict(scaler_values)
prediction

1/1 [==============================] - 0s 21ms/step


array([[0.02661898]], dtype=float32)

In [15]:
prediction_proba = prediction[0][0]

In [16]:
prediction_proba

0.026618978

In [17]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
